# Monitor and Govern Databricks Workspaces

Use system tables to monitor usage, costs, and implement governance with Unity Catalog.

## What You'll Learn

✅ Query system tables for observability  
✅ Analyze billing and cost allocation  
✅ Monitor workspace usage and performance  
✅ Implement Unity Catalog security  
✅ Create governance dashboards  

**Note**: Since students won't have access to actual system tables, we'll use synthetic data that matches the schema.

---

**References:**
- [System Tables](https://docs.databricks.com/aws/en/admin/system-tables/)
- [Billing Tables](https://docs.databricks.com/aws/en/admin/system-tables/billing)
- [Unity Catalog Governance](https://docs.databricks.com/aws/en/data-governance/unity-catalog/)
- [Observability Dashboards](https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources)

In [ ]:
%sql
-- Get your current username to use in queries below
-- Copy the part before @ and replace special characters with underscores

SELECT 
  current_user() as your_email,
  REGEXP_REPLACE(SPLIT(current_user(), '@')[0], '[^a-zA-Z0-9_]', '_') as your_schema_name;

-- 📋 INSTRUCTIONS:
-- 1. Run this cell to see YOUR username
-- 2. Copy the value from 'your_schema_name' column
-- 3. Use it to replace {your_username} in the queries below
-- 
-- Example: If your_schema_name shows 'jane_smith', replace:
--   dwx_airops_insights_platform_dev_working.{your_username}
-- with:
--   dwx_airops_insights_platform_dev_working.jane_smith

## Cost Analysis with System Tables

In production environments, Databricks provides **system tables** in the `system.*` catalog for monitoring usage and costs. For this training, we've created synthetic system tables in the shared schema that match the production schema.

**Note:** All queries below can be run directly on a SQL Warehouse. Simply replace `{your_username}` with your schema name from the cell above.

### Total Cost by Day

Analyze daily cost trends to understand spending patterns:

In [ ]:
%sql
-- Total Cost by Day
SELECT 
  usage_date,
  SUM(usage_quantity * list_price) as total_cost,
  COUNT(*) as num_operations
FROM dwx_airops_insights_platform_dev_working.db_crash_course.system_billing
GROUP BY usage_date
ORDER BY usage_date DESC
LIMIT 10

In [ ]:
%sql
-- Cost by Workspace
SELECT 
  workspace_id,
  SUM(usage_quantity * list_price) as total_cost,
  COUNT(*) as num_operations,
  AVG(usage_quantity * list_price) as avg_cost_per_operation
FROM dwx_airops_insights_platform_dev_working.db_crash_course.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY workspace_id
ORDER BY total_cost DESC

In [ ]:
%sql
-- Cost by SKU Type
SELECT 
  sku_name,
  SUM(usage_quantity) as total_dbus,
  SUM(usage_quantity * list_price) as total_cost,
  AVG(usage_quantity * list_price) as avg_cost_per_operation,
  COUNT(*) as operations
FROM dwx_airops_insights_platform_dev_working.db_crash_course.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY sku_name
ORDER BY total_cost DESC

In [ ]:
%sql
-- Cost by User
SELECT 
  usage_metadata.user,
  COUNT(*) as operations,
  SUM(usage_quantity * list_price) as total_cost,
  AVG(usage_quantity * list_price) as avg_cost_per_operation
FROM dwx_airops_insights_platform_dev_working.db_crash_course.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY usage_metadata.user
ORDER BY total_cost DESC
LIMIT 10

In [ ]:
%sql
-- Consolidated query performance summary
SELECT 
  query_type,
  COUNT(*) as query_count,
  AVG(execution_time_ms) as avg_duration_ms,
  AVG(compute_cost) as avg_cost,
  SUM(compute_cost) as total_cost
FROM dwx_airops_insights_platform_dev_working.db_crash_course.query_history
WHERE query_start_time >= CURRENT_DATE - 7
GROUP BY query_type
ORDER BY total_cost DESC

In [0]:
# Cost by SKU Type
spark.sql(f"""
SELECT 
  sku_name,
  SUM(usage_quantity) as total_dbus,
  SUM(usage_quantity * list_price) as total_cost,
  AVG(usage_quantity * list_price) as avg_cost_per_operation,
  COUNT(*) as operations
FROM {CATALOG}.{READ_SCHEMA}.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY sku_name
ORDER BY total_cost DESC
""").display()

In [0]:
# Cost by User
spark.sql(f"""
SELECT 
  usage_metadata.user,
  COUNT(*) as operations,
  SUM(usage_quantity * list_price) as total_cost,
  AVG(usage_quantity * list_price) as avg_cost_per_operation
FROM {CATALOG}.{READ_SCHEMA}.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY usage_metadata.user
ORDER BY total_cost DESC
LIMIT 10
""").display()

In [0]:
# Consolidated query performance summary
spark.sql(f"""
SELECT 
  query_type,
  COUNT(*) as query_count,
  AVG(execution_time_ms) as avg_duration_ms,
  AVG(compute_cost) as avg_cost,
  SUM(compute_cost) as total_cost
FROM {CATALOG}.{READ_SCHEMA}.query_history
WHERE query_start_time >= CURRENT_DATE - 7
GROUP BY query_type
ORDER BY total_cost DESC
""").display()

In [0]:
spark.sql(f"""
-- Consolidated query performance summary
SELECT 
  query_type,
  COUNT(*) as query_count,
  AVG(execution_time_ms) as avg_duration_ms,
  AVG(compute_cost) as avg_cost,
  SUM(compute_cost) as total_cost
FROM {CATALOG}.{READ_SCHEMA}.query_history
WHERE query_start_time >= CURRENT_DATE - 7
GROUP BY query_type
ORDER BY total_cost DESC;
""")

**Key Insights:**
- Identify which query types are driving costs
- Spot performance optimization opportunities
- Track usage patterns across your team

For more detailed analysis, explore the [Observability Dashboards](https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources) examples.


### Use Cases for AI Forecast

**Cost Management:**
- Predict monthly spending for budget planning
- Identify cost spikes before they happen
- Allocate resources based on forecasted demand

**Capacity Planning:**
- Forecast compute usage by team/project
- Plan infrastructure scaling
- Optimize reserved capacity purchases

**Reference:** [AI Forecast Documentation](https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_forecast)

**💡 Pro Tip:** Create a scheduled job that runs these forecasts daily and sends alerts when predicted costs exceed thresholds.



In [0]:
%sql
-- Forecast daily costs for the next 7 days using AI
-- Note: This requires a SQL Warehouse connection

SELECT 
  usage_date,
  total_cost,
  forecast,
  lower_bound,
  upper_bound
FROM (
  SELECT 
    ai_forecast(
      (SELECT usage_date, SUM(usage_quantity * list_price) as total_cost
       FROM {CATALOG}.{READ_SCHEMA}
       GROUP BY usage_date
       ORDER BY usage_date),
      horizon => 7
    )
  )
ORDER BY usage_date DESC


# Unity Catalog Access Management

One of the most important aspects of data governance is controlling who can access what data. Unity Catalog provides fine-grained access control at multiple levels: catalogs, schemas, tables, columns, and even rows.

Let's walk through a practical example of setting up access controls for our IoT sensor data.


### Step 1: Create Groups

Groups make it easier to manage permissions at scale. Instead of granting access to individual users, you grant it to groups.

**Common groups for an IoT project:**
- `data_engineers` - Can read/write raw and processed data
- `data_analysts` - Can read processed data and create dashboards
- `ml_engineers` - Can read data and create/deploy ML models
- `executives` - Read-only access to dashboards and reports


In [0]:
%sql
-- Create a group for data analysts
-- Note: In Databricks, groups are typically managed at the account level
-- These commands would be run by an account admin

CREATE GROUP IF NOT EXISTS `data_analysts_xyz`; -- You will hit an error later if you don't create your own group


### Step 2: Add Users to Groups

Once groups are created, you can add users to them. Users inherit all permissions granted to the groups they belong to.


In [0]:
%sql
-- Add users to groups
-- Replace with actual user emails from your organization

ALTER GROUP `data_analysts_xyz` ADD USER `jane.smith@company.com`; --make sure you use your own group created above, and add a real user

### Step 3: Grant Table Access

Now let's grant the appropriate permissions on our IoT tables. Unity Catalog uses a hierarchical permission model:

**Hierarchy:** Catalog → Schema → Table

**Permission Types:**
- `USE CATALOG` - Required to see/access a catalog
- `USE SCHEMA` - Required to see/access a schema
- `SELECT` - Read data from tables
- `MODIFY` - Update/delete data
- `CREATE TABLE` - Create new tables in a schema
- `ALL PRIVILEGES` - Full control

Let's grant permissions for our sensor and inspection data:


In [0]:
%sql
-- First, grant catalog and schema level permissions
-- Data analysts need to USE the catalog and schema to access tables
GRANT USE CATALOG ON CATALOG dwx_airops_insights_platform_dev_working TO `data_analysts_xyz`;
GRANT USE SCHEMA ON SCHEMA dwx_airops_insights_platform_dev_working.{your_username} TO `data_analysts_xyz`;
GRANT SELECT ON TABLE dwx_airops_insights_platform_dev_working.{your_username}.sensor_bronze TO `data_analysts_xyz`;

## Advanced Security Features (Illustrative Examples - Do Not Run)

Unity Catalog supports advanced security features for fine-grained access control. The examples below are **for reference only** to illustrate what's possible in production environments. **These commands are not meant to be executed in this training** - they require additional setup and permissions typically managed by administrators.

### Row-Level Security (Reference Pattern)

**What it does:** Restricts which rows users can see based on their identity or group membership.

**Example use case:** Regional managers only see data for their region, while executives see all regions.

**Illustrative SQL pattern** (do not run):
```sql
-- Step 1: Create a filter function that checks user permissions
CREATE FUNCTION your_catalog.your_schema.filter_by_region(region STRING)
RETURN region IN (
  SELECT region FROM your_catalog.your_schema.user_permissions 
  WHERE user_email = current_user()
);

-- Step 2: Apply the filter to a table
ALTER TABLE your_catalog.your_schema.sensor_data
SET ROW FILTER your_catalog.your_schema.filter_by_region(region) ON (region);
```

**Result:** When any user queries `sensor_data`, they automatically only see rows for regions they're authorized to access. The filter is transparent to the user.

---

### Column Masking (Reference Pattern)

**What it does:** Shows different column values based on user permissions, masking sensitive data for unauthorized users.

**Example use case:** Analysts see masked device IDs (`***1234`) while engineers with elevated permissions see full IDs.

**Illustrative SQL pattern** (do not run):
```sql
-- Step 1: Create masking function based on user role
CREATE FUNCTION your_catalog.your_schema.mask_device_id(device_id STRING)
RETURN CASE 
  WHEN is_member('admin') THEN device_id
  ELSE CONCAT('***', RIGHT(device_id, 4))
END;

-- Step 2: Apply mask to a column
ALTER TABLE your_catalog.your_schema.sensor_data
ALTER COLUMN device_id
SET MASK your_catalog.your_schema.mask_device_id;
```

**Result:** Non-admin users automatically see masked device IDs (e.g., "***1234") in all queries, while admins see the full values.

---

### When You'd Use These Features

These advanced security patterns are typically implemented by data governance teams in production environments when:
- Compliance regulations require data access restrictions (GDPR, HIPAA, etc.)
- Different business units need access to the same tables but different rows
- Sensitive data must be masked for certain user groups
- Audit requirements mandate automatic, transparent security controls

**To learn more about implementing these in your production environment:**
- [Row Filters and Column Masks Documentation](https://docs.databricks.com/en/data-governance/unity-catalog/row-and-column-filters)
- [Unity Catalog Security Best Practices](https://docs.databricks.com/en/data-governance/unity-catalog/best-practices)
- [Implementing Data Governance](https://docs.databricks.com/en/data-governance/)

In [ ]:
%sql
-- View all grants on a specific table
SHOW GRANTS ON TABLE dwx_airops_insights_platform_dev_working.{your_username}.sensor_bronze;

-- View all grants for a specific group
-- SHOW GRANTS TO `data_analysts_xyz`;

-- View grants on an entire schema
-- SHOW GRANTS ON SCHEMA dwx_airops_insights_platform_dev_working.{your_username};

### Revoking Permissions

If you need to remove access, use the `REVOKE` command:

```sql
-- Revoke SELECT on a specific table
REVOKE SELECT ON TABLE dwx_airops_insights_platform_dev_working.{your_username}.sensor_bronze FROM `data_analysts_xyz`;

-- Revoke all privileges on a schema
REVOKE ALL PRIVILEGES ON SCHEMA dwx_airops_insights_platform_dev_working.{your_username} FROM `data_analysts_xyz`;
```

**Best Practices for Access Management:**

1. **Use groups, not individual users** - Makes management much easier
2. **Grant minimum necessary permissions** - Start with read-only, add write permissions only when needed
3. **Use separate schemas for different purposes** - e.g., `raw`, `processed`, `ml_features`, `production`
4. **Document your permission model** - Keep track of which groups have access to what
5. **Regular audits** - Review permissions quarterly to ensure they're still appropriate
6. **Use service principals for automation** - Don't use personal accounts for scheduled jobs

## Advanced: Row-Level Security and Column Masking (Optional)

Unity Catalog supports advanced security features for fine-grained access control. These are conceptual examples to understand the patterns - you can explore these in your own projects.

### Row-Level Security Concept

**What it does:** Restricts which rows users can see based on their identity or group membership.

**Use case:** East region managers can only see data for their region, while executives see all regions.

**Pattern:**
```sql
-- Step 1: Create a filter function in your schema
CREATE FUNCTION {your_catalog}.{your_schema}.filter_by_region(region STRING)
RETURN region IN (
  SELECT region FROM {your_catalog}.{your_schema}.user_permissions 
  WHERE user_email = current_user()
);

-- Step 2: Apply the filter to a table
ALTER TABLE {your_catalog}.{your_schema}.sensor_data
SET ROW FILTER {your_catalog}.{your_schema}.filter_by_region(region) ON (region);
```

Now when users query `sensor_data`, they automatically only see rows for their authorized regions.

---

### Column Masking Concept

**What it does:** Shows different column values based on user permissions (e.g., mask sensitive IDs).

**Use case:** Analysts see masked device IDs (`***1234`) while engineers see full IDs.

**Pattern:**
```sql
-- Step 1: Create masking function in your schema
CREATE FUNCTION {your_catalog}.{your_schema}.mask_device_id(device_id STRING)
RETURN CASE 
  WHEN is_member('admin') THEN device_id
  ELSE CONCAT('***', RIGHT(device_id, 4))
END;

-- Step 2: Apply mask to column
ALTER TABLE {your_catalog}.{your_schema}.sensor_data
ALTER COLUMN device_id
SET MASK {your_catalog}.{your_schema}.mask_device_id;
```

Non-admin users will only see masked device IDs (e.g., "***1234") when querying the table.

---

**To learn more:**
- [Row Filters and Column Masks Documentation](https://docs.databricks.com/en/data-governance/unity-catalog/row-and-column-filters)
- [Unity Catalog Security Best Practices](https://docs.databricks.com/en/data-governance/unity-catalog/best-practices)


---

**Additional Resources:**
- [System Tables Guide](https://docs.databricks.com/aws/en/admin/system-tables/)
- [Unity Catalog Security](https://docs.databricks.com/aws/en/data-governance/unity-catalog/access-control)
- [Observability Examples](https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources)
- [Cost Management](https://docs.databricks.com/aws/en/admin/account-settings/usage-detail-tags-aws)
- [Row Filters and Column Masks](https://docs.databricks.com/aws/en/data-governance/unity-catalog/row-and-column-filters)
- [AI Forecast Function](https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_forecast)

---

**🎉 You've completed Day 3!** You now have the skills to build end-to-end data and ML pipelines, monitor costs, forecast future spending with AI, and govern your Databricks workspace effectively.